# 📊 Telco Customer Churn Intelligence Platform (CCIP)
### End-to-End ML Project — Akeem Asiru

This notebook walks through the **full machine learning workflow** behind the Telco Customer Churn Intelligence Platform (CCIP):

- Business understanding & problem framing  
- Data loading & exploration  
- Cleaning & feature engineering  
- Model training & evaluation  
- Model interpretability (SHAP)  
- Saving the production model (`best_churn_model.joblib`)  
- Helper functions for **single** and **batch** predictions  

> **Author:** Akeem Asiru — Machine Learning Engineer  
> **Project:** Telco Customer Churn Intelligence Platform (CCIP)


## 1. Imports & Configuration

In [ ]:
import os
from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score,
    roc_auc_score,
    classification_report,
    confusion_matrix,
    RocCurveDisplay,
)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

import joblib

# Optional: SHAP for explainability
try:
    import shap
    SHAP_AVAILABLE = True
except ImportError:
    SHAP_AVAILABLE = False

# Plot style
plt.style.use("seaborn-v0_8")
sns.set_palette("Set2")

# Paths (assumes this notebook lives at project root)
PROJECT_ROOT = Path.cwd()
DATA_PATH = PROJECT_ROOT / "data" / "Telco-Customer-Churn.csv"
MODELS_DIR = PROJECT_ROOT / "models"
MODELS_DIR.mkdir(exist_ok=True)

print(f"Project root: {PROJECT_ROOT}")
print(f"Data path:    {DATA_PATH}")
print(f"Models dir:   {MODELS_DIR}")

## 2. Business Problem Framing

**Business question:**  
> _Which customers are most likely to churn in the next period, and how can we prioritize retention efforts to reduce churn and protect revenue?_

**Key goals:**  
- Predict the probability that a given customer will churn.  
- Quantify overall churn rate and segment-level churn.  
- Provide interpretable drivers of churn using SHAP.  
- Enable single-customer and batch predictions for downstream apps (FastAPI, Streamlit, dashboards).

## 3. Load Telco Churn Dataset

In [ ]:
df = pd.read_csv(DATA_PATH)
df.columns = df.columns.str.strip()  # clean any trailing spaces

print(df.shape)
df.head()

## 4. Exploratory Data Analysis (EDA)

In [ ]:
print("Columns:\n", df.columns.tolist())
print("\nInfo:")
print(df.info())

In [ ]:
# Churn distribution
churn_counts = df["Churn"].value_counts()
churn_rate = (df["Churn"] == "Yes").mean()

fig, ax = plt.subplots(figsize=(4,4))
churn_counts.plot(kind="bar", ax=ax)
ax.set_title(f"Churn Distribution (rate = {churn_rate:.2%})")
ax.set_ylabel("Count")
plt.show()

In [ ]:
# Numeric summary
df.describe(include=[np.number]).T

## 5. Data Cleaning & Feature Engineering

In [ ]:
# Clean TotalCharges (string with spaces for missing)
if "TotalCharges" in df.columns:
    df["TotalCharges"] = df["TotalCharges"].replace(" ", np.nan)
    df["TotalCharges"] = df["TotalCharges"].astype(float)
    mask_missing = df["TotalCharges"].isna()
    df.loc[mask_missing, "TotalCharges"] = (
        df.loc[mask_missing, "MonthlyCharges"] * df.loc[mask_missing, "tenure"].fillna(0)
    )

print("Missing values after TotalCharges fix:\n", df.isna().sum())

In [ ]:
# Create binary target
df["ChurnFlag"] = (df["Churn"] == "Yes").astype(int)

# Simple feature engineering
df["MonthlyTenureInteraction"] = df["MonthlyCharges"] * df["tenure"]
df["HasMultipleServices"] = (
    (df["PhoneService"] == "Yes").astype(int)
    + (df["InternetService"] != "No").astype(int)
)

# Drop identifiers
drop_cols = ["customerID"]
for c in drop_cols:
    if c in df.columns:
        df = df.drop(columns=[c])

target_col = "ChurnFlag"
X = df.drop(columns=[target_col])
y = df[target_col]

print("Features shape:", X.shape)
print("Churn rate:", y.mean())

## 6. Feature Types (Numeric vs Categorical)

In [ ]:
numeric_features = X.select_dtypes(include=["int64", "float64"]).columns.tolist()
categorical_features = X.select_dtypes(include=["object", "bool"]).columns.tolist()

print("Numeric features (", len(numeric_features), "):\n", numeric_features)
print("\nCategorical features (", len(categorical_features), "):\n", categorical_features)

## 7. Preprocessing Pipeline (ColumnTransformer)

In [ ]:
numeric_transformer = Pipeline(
    steps=[("scaler", StandardScaler())]
)

categorical_transformer = Pipeline(
    steps=[("onehot", OneHotEncoder(handle_unknown="ignore"))]
)

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

preprocess

## 8. Train / Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train:", X_train.shape, "Test:", X_test.shape)

## 9. Define Candidate Models

In [ ]:
log_reg_clf = Pipeline(
    steps=[
        ("preprocess", preprocess),
        ("model", LogisticRegression(max_iter=1000, n_jobs=-1)),
    ]
)

rf_clf = Pipeline(
    steps=[
        ("preprocess", preprocess),
        ("model", RandomForestClassifier(
            n_estimators=300,
            random_state=42,
            n_jobs=-1
        )),
    ]
)

models = {
    "Logistic Regression": log_reg_clf,
    "Random Forest": rf_clf,
}

models

## 10. Train & Evaluate Models

In [ ]:
results_summary = []

for name, pipe in models.items():
    print("\n==============================")
    print(f"Training model: {name}")
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    y_proba = pipe.predict_proba(X_test)[:, 1]

    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba)

    print(f"Accuracy: {acc:.4f}")
    print(f"ROC AUC:  {auc:.4f}")
    print("\nClassification Report:\n", classification_report(y_test, y_pred))

    cm = confusion_matrix(y_test, y_pred)
    fig, ax = plt.subplots(figsize=(4, 4))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax)
    ax.set_title(f"Confusion Matrix — {name}")
    ax.set_xlabel("Predicted")
    ax.set_ylabel("True")
    plt.show()

    RocCurveDisplay.from_predictions(y_test, y_proba)
    plt.title(f"ROC Curve — {name}")
    plt.show()

    results_summary.append({
        "model": name,
        "accuracy": acc,
        "roc_auc": auc,
    })

results_df = pd.DataFrame(results_summary).sort_values("roc_auc", ascending=False)
results_df

## 11. Select Best Model & Save to `models/best_churn_model.joblib`

In [ ]:
best_row = results_df.iloc[0]
best_name = best_row["model"]
print("\n🏆 Best model:", best_name)

best_model = models[best_name]

# Optionally refit on full data (train + test)
best_model.fit(X, y)

model_path = MODELS_DIR / "best_churn_model.joblib"
joblib.dump(best_model, model_path)

print(f"✅ Saved best model pipeline to: {model_path}")

## 12. Single-Customer Prediction Helper

In [ ]:
example_customer = {
    "gender": "Female",
    "SeniorCitizen": 0,
    "Partner": "Yes",
    "Dependents": "No",
    "tenure": 5,
    "PhoneService": "Yes",
    "MultipleLines": "No",
    "InternetService": "Fiber optic",
    "OnlineSecurity": "No",
    "OnlineBackup": "No",
    "DeviceProtection": "No",
    "TechSupport": "No",
    "StreamingTV": "Yes",
    "StreamingMovies": "Yes",
    "Contract": "Month-to-month",
    "PaperlessBilling": "Yes",
    "PaymentMethod": "Electronic check",
    "MonthlyCharges": 80.5,
    "TotalCharges": 400.0,
    "MonthlyTenureInteraction": 80.5 * 5,
    "HasMultipleServices": 2,
}

example_df = pd.DataFrame([example_customer])
proba = best_model.predict_proba(example_df)[0, 1]
label = "Churn" if proba >= 0.5 else "No Churn"

print(f"Predicted churn probability: {proba:.3f}")
print(f"Predicted label: {label}")

## 13. Batch Prediction Helper (for CSV Files)

In [ ]:
def run_batch_prediction(input_df: pd.DataFrame) -> pd.DataFrame:
    """Run churn predictions on a batch of customers.
    Expects raw Telco-like schema, including:
    - tenure, MonthlyCharges, TotalCharges
    - service features
    - but will ignore 'customerID' and 'Churn' if present.
    """
    df_batch = input_df.copy()

    # Clean TotalCharges
    if "TotalCharges" in df_batch.columns:
        df_batch["TotalCharges"] = df_batch["TotalCharges"].replace(" ", np.nan)
        df_batch["TotalCharges"] = df_batch["TotalCharges"].astype(float)
        mask_missing = df_batch["TotalCharges"].isna()
        df_batch.loc[mask_missing, "TotalCharges"] = (
            df_batch.loc[mask_missing, "MonthlyCharges"] * df_batch.loc[mask_missing, "tenure"].fillna(0)
        )

    # Feature engineering
    df_batch["MonthlyTenureInteraction"] = df_batch["MonthlyCharges"] * df_batch["tenure"].fillna(0)
    df_batch["HasMultipleServices"] = (
        (df_batch["PhoneService"] == "Yes").astype(int)
        + (df_batch["InternetService"] != "No").astype(int)
    )

    # Drop unwanted columns if present
    for col in ["Churn", "ChurnFlag", "customerID"]:
        if col in df_batch.columns:
            df_batch = df_batch.drop(columns=[col])

    # Predict
    proba = best_model.predict_proba(df_batch)[:, 1]
    pred = (proba >= 0.5).astype(int)

    out_df = df_batch.copy()
    out_df["churn_prediction"] = pred
    out_df["churn_probability"] = proba

    return out_df

# Quick smoke test on first 5 rows (dropping target)
batch_test = run_batch_prediction(df.drop(columns=["Churn", "ChurnFlag"], errors="ignore").head(5))
batch_test

## 14. Model Explainability with SHAP (Optional)

In [ ]:
if SHAP_AVAILABLE:
    print("SHAP is available — computing global feature importance on a sample...")
    preprocess_step = best_model.named_steps["preprocess"]
    model_step = best_model.named_steps["model"]

    X_sample = X.sample(300, random_state=42)
    X_trans = preprocess_step.transform(X_sample)

    # Use TreeExplainer for tree-based models, otherwise generic Explainer
    if isinstance(model_step, RandomForestClassifier):
        explainer = shap.TreeExplainer(model_step)
        shap_values = explainer(X_trans)
    else:
        explainer = shap.Explainer(model_step, X_trans)
        shap_values = explainer(X_trans)

    shap.summary_plot(shap_values, X_trans, show=True)
else:
    print("SHAP is not installed. Run `pip install shap` to enable explainability plots.")

## 15. Next Steps & Integration

This notebook produces a production-ready model artifact:

- `models/best_churn_model.joblib`

This model is:
- Loaded by the **FastAPI** backend for real-time scoring.  
- Used by the **Streamlit** dashboard to power:
  - Single customer predictions  
  - Batch predictions  
  - Advanced KMeans segmentation  
  - Churn risk insights  

**Deployed system components (outside of this notebook):**
- FastAPI model-serving API  
- Streamlit executive dashboard  
- Prometheus & Grafana for monitoring  
- Docker + Nginx for containerized deployment  
- (Planned) GitHub Actions CI/CD for auto-deploys to EC2  

> This notebook is the **analytical heart** of the Telco Customer Churn Intelligence Platform (CCIP) built by **Akeem Asiru**.
